# 💻 Laptop Price Prediction - Kaggle
Pipeline completo con feature engineering y modelo optimizado.

## 📦 1. Librerías

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor

## 📂 2. Cargar datos

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

## 🎯 3. Separar variable objetivo

In [ ]:
y = train["Price_in_euros"]

X = train.drop(columns=["Price_in_euros", "laptop_ID"])
X_test = test.drop(columns=["laptop_ID"])

## 🧹 4. Limpieza básica

In [ ]:
for df in [X, X_test]:
    df["Ram"] = df["Ram"].str.replace("GB", "").astype(int)
    df["Weight"] = df["Weight"].str.replace("kg", "").astype(float)

## ⚙️ 5. Feature Engineering

In [ ]:
def extract_ghz(cpu):
    match = re.search(r'(\d+\.?\d*)GHz', cpu)
    return float(match.group(1)) if match else np.nan

for df in [X, X_test]:
    df["CPU_GHz"] = df["Cpu"].apply(extract_ghz)
    df["CPU_brand"] = df["Cpu"].apply(lambda x: x.split()[0])
    df["SSD"] = df["Memory"].str.contains("SSD").astype(int)
    df["Memory_size"] = df["Memory"].str.extract(r'(\d+)').astype(float)
    df["FullHD"] = df["ScreenResolution"].str.contains("1920x1080").astype(int)

X = X.drop(columns=["Cpu", "Memory", "ScreenResolution"])
X_test = X_test.drop(columns=["Cpu", "Memory", "ScreenResolution"])

## 🏷️ 6. One Hot Encoding

In [ ]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

X, X_test = X.align(X_test, join="left", axis=1, fill_value=0)

## 🤖 7. Modelo

In [ ]:
model = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)

model.fit(X, y)

## 🔮 8. Predicción

In [ ]:
preds = model.predict(X_test)

## 📄 9. Crear submission

In [ ]:
submission = pd.DataFrame({
    "laptop_ID": test["laptop_ID"],
    "Price_in_euros": preds
})

submission.to_csv("submission.csv", index=False)

submission.head()